# Quality assurance and automated service data review

This notebook reviews published service data for common mistakes. Relies on gc-service-data-script outputs to function.

In [1]:
import pandas as pd
import numpy as np
import requests, pytz, os, re
from tqdm.notebook import tqdm  # Use tqdm.notebook for JupyterLab progress bars
from concurrent.futures import ThreadPoolExecutor, as_completed


In [2]:
# Specify date and time in correct timezone
timezone = pytz.timezone('America/Montreal')
current_datetime = pd.Timestamp.now(tz=timezone)
current_datetime_str = current_datetime.strftime("%Y-%m-%d_%H:%M:%S")
print(f'Current datetime: {current_datetime_str}')

Current datetime: 2024-12-10_11:26:19


In [3]:
# Import service inventory and service standards to dataframes
si = pd.read_csv('si.csv', sep=';',  na_values=[], keep_default_na=False)
ss = pd.read_csv('ss.csv', sep=';',  na_values=[], keep_default_na=False)
qa_issues_description = pd.read_csv('qa_issues_descriptions.csv')

# Extract date of generation from timestamp on last line
date = pd.to_datetime(si.iloc[-1, 0].split(':')[1].split('_')[0])

# Remove last line with datestamp from dataframes
si = si.iloc[:-1]
ss = ss.iloc[:-1]

In [4]:
# Coerce all numeric fields
int_cols = {
    'num_phone_enquiries': si,
    'num_applications_by_phone': si,
    'num_website_visits': si,
    'num_applications_online': si,
    'num_applications_by_mail': si,
    'num_applications_by_email': si,
    'num_applications_by_fax': si,
    'num_applications_by_other': si,
    'num_applications_total': si,
    'volume_meeting_target': ss,
    'total_volume': ss
}

# int_cols[column][column]
# is int_cols[column] = dict[key],
# then dict[key] = dataframe, 
# so dict[key][column] = dataframe[column]

for column, df in int_cols.items():
    int_cols[column][column] = pd.to_numeric(df[column], errors = 'coerce').fillna(0).astype(int)

In [5]:
# Duplicate service ID conflict
# Step 1: Flag rows where 'service_id' is duplicated within each 'fiscal_yr'
si['qa_duplicate_sid'] = si.duplicated(subset=['fiscal_yr', 'service_id'], keep=False)

# Step 2: Get unique 'service_id's that are flagged as duplicates
duplicate_ids = si.loc[si['qa_duplicate_sid'], 'service_id'].unique()

# Step 3: Filter rows with duplicate 'service_id's and group by 'service_id' and 'department_en'
duplicate_groups = (
    si.loc[si['service_id'].isin(duplicate_ids), ['fiscal_yr', 'service_id', 'department_en']]
    .groupby(['service_id', 'department_en'])['fiscal_yr']  # Count occurrences of 'fiscal_yr'
    .nunique()  # Count unique fiscal years for each group
)

# Step 4: Identify groups with only one unique fiscal year (problematic cases)
problematic_duplicates = duplicate_groups[duplicate_groups == 1].reset_index()

# Step 5: Keep only 'service_id' and 'department_en' columns
problematic_duplicates = problematic_duplicates[['service_id', 'department_en']]

# Step 6: Create a set of tuples from 'problematic_duplicates' for efficient lookup
problematic_set = set(zip(problematic_duplicates['service_id'], problematic_duplicates['department_en']))

# Step 7: Update the 'qa_duplicate_sid' column based on whether each row matches a problematic duplicate
si['qa_duplicate_sid'] = si.apply(
    lambda row: (row['service_id'], row['department_en']) in problematic_set, axis=1
)


# check:
# si.loc[:, ['fiscal_yr', 'department_en', 'service_id', 'qa_duplicate_sid']][si['qa_duplicate_sid']]

In [6]:
# Duplicate Service Standard ID conflict
# Step 1: Flag rows where 'service_standard_id' is duplicated within each 'fiscal_yr'
ss['qa_duplicate_stdid'] = ss.duplicated(subset=['fiscal_yr', 'service_standard_id'], keep=False)

# Step 2: Get unique 'service_standard_id's that are flagged as duplicates
duplicate_ids = ss.loc[ss['qa_duplicate_stdid'], 'service_standard_id'].unique()

# Step 3: Filter rows with duplicate 'service_standard_id's and group by 'service_standard_id' and 'department_en'
duplicate_groups = (
    ss.loc[ss['service_standard_id'].isin(duplicate_ids), ['fiscal_yr', 'service_standard_id', 'department_en']]
    .groupby(['service_standard_id', 'department_en'])['fiscal_yr']  # Count occurrences of 'fiscal_yr'
    .nunique()  # Count unique fiscal years for each group
)

# Step 4: Identify groups with only one unique fiscal year (problematic cases)
problematic_duplicates = duplicate_groups[duplicate_groups == 1].reset_index()

# Step 5: Keep only 'service_standard_id' and 'department_en' columns
problematic_duplicates = problematic_duplicates[['service_standard_id', 'department_en']]

# Step 6: Create a set of tuples from 'problematic_duplicates' for efficient lookup
problematic_set = set(zip(problematic_duplicates['service_standard_id'], problematic_duplicates['department_en']))

# Step 7: Update the 'qa_duplicate_sid' column based on whether each row matches a problematic duplicate
ss['qa_duplicate_stdid'] = ss.apply(
    lambda row: (row['service_standard_id'], row['department_en']) in problematic_set, axis=1
)


# check:
# ss.loc[:, ['fiscal_yr', 'department_en', 'service_id', 'service_standard_id', 'qa_duplicate_stdid']][ss['qa_duplicate_stdid']]

In [7]:
# Record is reported for a fiscal year that is incomplete or in the future.
si['fiscal_yr_end_date'] = pd.to_datetime(si['fiscal_yr'].str.split('-').str[1]+'-04-01')
si['qa_si_fiscal_yr_in_future'] = si['fiscal_yr_end_date'] >= date

ss['fiscal_yr_end_date'] = pd.to_datetime(ss['fiscal_yr'].str.split('-').str[1]+'-04-01')
ss['qa_ss_fiscal_yr_in_future'] = ss['fiscal_yr_end_date'] >= date

In [8]:
# Record has contradiction between client feedback channels and online interaction points for feedback
si['qa_client_feedback_contradiction'] = (

    # Service accepts client feedback via the online channel (ONL) but online issue resolution or feedback is not applicable or not activated
    (
        si['client_feedback_channel'].str.contains('ONL') & 
        (
            si['os_issue_resolution_feedback'].isna() | 
            (si['os_issue_resolution_feedback'] == 'N')
        )
    ) |
    # Service has not listed the online channel (ONL) for client feedback but online issue resolution or feedback is activated
    (
        (~si['client_feedback_channel'].str.contains('ONL')) &
        (si['os_issue_resolution_feedback'] == 'Y')
    )
)

# si[['client_feedback_channel', 'os_issue_resolution_feedback', 'client_feedback_contradiction']].loc[si['client_feedback_contradiction'] == True]

In [9]:
# Service standards have volume but no volume indicated at service level
ss_vol_by_service = (
    ss.groupby(['fiscal_yr', 'service_id'])['total_volume']
    .sum()
    .reset_index()
    .rename(columns={'total_volume':'total_volume_ss'})
)

si = si.merge(ss_vol_by_service, on=['fiscal_yr', 'service_id'], how='left').fillna(0)

si['qa_ss_vol_without_si_vol'] = (
    (si['total_volume_ss'] > 0) & (si['num_applications_total'] == 0)
)

In [10]:
# Service standard reports no volume
ss['qa_no_ss_volume'] = (ss['total_volume'] == 0)

In [11]:
# Services that target society as a recipient type we would not expect to see specific interaction volume
# Note that this assumption may be false
si['num_applications_total'] = pd.to_numeric(si['num_applications_total'], errors = 'coerce').fillna(0).astype(int)

si['qa_service_recipient_type_society_with_interactions'] = (
    (si['service_recipient_type'] == 'SOCIETY') &
    (si['num_applications_total'] > 0)
)

In [12]:
# Services where 'persons' are a client type should not be 'NA' for SIN as ID
si['qa_use_of_sin_applicable'] = (
    (si['client_target_groups'].str.contains('PERSON')) &
    (si['sin_usage'] == 'NA')
)

In [13]:
# Services where 'econom' (business) are a client type should not be 'NA' for CRA BN as ID
si['qa_use_of_cra_bn_applicable'] = (
    (si['client_target_groups'].str.contains('ECONOM')) &
    (si['cra_bn_identifier_usage'] == 'NA')
)

### Clean QA report

In order to have a clean report of issues to send to departments & agencies, the following bit of script re-organizes the information in the qa columns to a simple report for 2023-2024 data.

In [14]:
si_qa_cols = si.columns.str.startswith('qa')
ss_qa_cols = ss.columns.str.startswith('qa')

critical_si_qa_cols = [
    'qa_duplicate_sid',
    'qa_si_fiscal_yr_in_future',
    'qa_ss_vol_without_si_vol',
]

critical_ss_qa_cols = [
    'qa_duplicate_stdid',
    'qa_no_ss_volume',
    'qa_ss_fiscal_yr_in_future'
]

In [15]:
# Preparing SI QA report
si_report_cols = [
    'department_en',
    'fiscal_yr', 
    'service_id', 
    'service_name_en', 
    'service_name_fr',
    'num_applications_total',
    'total_volume_ss'
]

si_qa_report = pd.melt(si, id_vars=si_report_cols, value_vars=critical_si_qa_cols, var_name='issue', value_name='issue_present')

si_qa_report = si_qa_report[(si_qa_report['issue_present'] & si_qa_report['fiscal_yr'].isin(['2023-2024', '2024-2025']))]

si_qa_report = pd.merge(si_qa_report, qa_issues_description.loc[:, ['qa_field_name', 'description_en', 'action_en']], left_on='issue', right_on='qa_field_name', how='left')

si_qa_report = si_qa_report.drop(columns=['issue_present', 'qa_field_name'])

si_qa_report = si_qa_report.sort_values(by=['department_en', 'service_id'])

In [16]:
# Preparing SI QA report
ss_report_cols = [
    'department_en',
    'fiscal_yr', 
    'service_id', 
    'service_name_en', 
    'service_name_fr',
    'service_standard_id',
    'service_standard_en',
    'service_standard_fr',
    'volume_meeting_target',
    'total_volume',
]

ss_qa_report = pd.melt(ss, id_vars=ss_report_cols, value_vars=critical_ss_qa_cols, var_name='issue', value_name='issue_present')

ss_qa_report = ss_qa_report[(ss_qa_report['issue_present'] & ss_qa_report['fiscal_yr'].isin(['2023-2024', '2024-2025']))]

ss_qa_report = pd.merge(ss_qa_report, qa_issues_description.loc[:, ['qa_field_name', 'description_en', 'action_en']], left_on='issue', right_on='qa_field_name', how='left')

ss_qa_report = ss_qa_report.drop(columns=['issue_present', 'qa_field_name'])

ss_qa_report = ss_qa_report.sort_values(by=['department_en', 'service_id', 'service_standard_id'])

## Export data to CSV

In [17]:
# Define the DataFrames to export to csv and their corresponding names
csv_exports = {
    "si_qa": si,
    "ss_qa": ss,
    "si_qa_report": si_qa_report,
    "ss_qa_report": ss_qa_report
}

# Loop through the dictionary
for name, df in csv_exports.items():
    # Generate the filename using the key (string name)
    fn = f"{name}.csv"
    
    # Export the DataFrame to CSV
    df.to_csv(fn, index=False, sep=';')
    
    # Append the timestamp at the end of the file
    with open(fn, 'a') as timestamped_file:
        timestamped_file.write(f"\nTimestamp:{current_datetime_str}\n")